In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

from diffusers.models.autoencoders.autoencoder_dc import AutoencoderDC

In [3]:
from flow_flowers import Config, find_and_chdir

In [4]:
torch.manual_seed(42)
find_and_chdir("config.yaml")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
config = Config.init("config.yaml")

In [6]:
vae = AutoencoderDC.from_pretrained(config.model.autoencoder.id)
vae

AutoencoderDC(
  (encoder): Encoder(
    (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (down_blocks): ModuleList(
      (0): Sequential(
        (0): ResBlock(
          (nonlinearity): SiLU()
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (norm): RMSNorm()
        )
        (1): ResBlock(
          (nonlinearity): SiLU()
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (norm): RMSNorm()
        )
        (2): DCDownBlock2d(
          (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        )
      )
      (1): Sequential(
        (0): ResBlock(
          (nonlinearity): SiLU()
          (conv1): Conv2d(256, 256, kernel_size=(3, 3), s